<a href="https://colab.research.google.com/github/kgreed4/no_hate_transformer/blob/kgreed/bert_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# Load the dataset
df = pd.read_csv('cleaned_data_sw.csv')

# Drop every column that isn't tweet or class
df = df.drop(df.columns.difference(['tweet', 'class']), axis=1)

# First, we want to use the BERT tokenizer to tokenize and encode the dataset into embeddings
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# We will create a small function that will take care of tokenization and encoding
def encode_texts(tokenizer, texts, max_length):
    encoding = tokenizer.batch_encode_plus(
        texts,
        # This is required to add special tokens such as the [CLS] and [SEP] tokens that indicate the start and end of a sentence
        add_special_tokens=True,
        # Here the padding variable is responsible for padding the sequences to the same length
        padding='max_length',
        # The max length of the tokenized sequences
        max_length=max_length,
        return_attention_mask=True,
        # Here we specify that we want the output to be TensorFlow tensors
        return_tensors='tf',
        # If the sequence is longer than max_length, it will be truncated to a fixed length
        truncation=True
    )
    # The encoding['input_ids'] contains the tokenized sequences
    # The encoding['attention_mask'] contains the attention masks and tells the model which tokens to pay attention to and which ones to ignore (mask token)
    return encoding['input_ids'], encoding['attention_mask']

# Here we define the maximum length (randomly chosen per ChatGPT's recommendation)
max_length = 128

# We can then call the function to tokenize and encode the dataset
input_ids, attention_masks = encode_texts(tokenizer, df['tweet'].tolist(), max_length)

# Here we create labels from the 'class' column
# This is the target variable that we want to predict
labels = tf.convert_to_tensor(df['class'].values, dtype=tf.int32).numpy()

# For some reason, I was getting an error saying that I needed to convert to NumPy arrays instead of TensorFlow tensors
# So I converted the input_ids and attention_masks to NumPy arrays
input_ids_np = input_ids.numpy()
attention_masks_np = attention_masks.numpy()

# We can then use the train_test_split function from scikit-learn to split the dataset into training and validation sets
train_inputs_np, validation_inputs_np, train_labels_np, validation_labels_np = train_test_split(input_ids_np, labels, random_state=2021, test_size=0.1)
train_masks_np, validation_masks_np, _, _ = train_test_split(attention_masks_np, labels, random_state=2021, test_size=0.1)

# Here the BUFFER_SIZE is the number of training inputs
BUFFER_SIZE = len(train_inputs_np)
# The batch size is 32
BATCH_SIZE = 32

# We can then take the NumPy arrays and convert them to TensorFlow datasets for both the training and validation sets
train_dataset = tf.data.Dataset.from_tensor_slices(((train_inputs_np, train_masks_np), train_labels_np)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
validation_dataset = tf.data.Dataset.from_tensor_slices(((validation_inputs_np, validation_masks_np), validation_labels_np)).batch(BATCH_SIZE)

In [35]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=2):
    train_losses = []
    train_accuracies = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0
        for inputs, attention_masks, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs, attention_mask=attention_masks)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs.logits, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = correct_predictions / total_predictions

        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_accuracy)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

    return model, train_losses, train_accuracies

In [ ]:
# Model parameters
num_layers = 4
d_model = 128
num_heads = 8
d_ff = 512
input_vocab_size = tokenizer.vocab_size + 2
maximum_position_encoding = 512
rate = 0.1
num_classes = df['class'].nunique()

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Fine-tune BERT model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

# Train the model
model, train_losses, train_accuracies = train_model(model, train_loader, criterion, optimizer, num_epochs=2)

import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(train_accuracies)
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(train_losses)
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
